## ECE-505 Computer Project 1
### Due Date Nov-6 2018

Step 1: Importing necessary libraries for all the necessary modules


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

### PART1-Algorithmic Implementation
#### 1.All the Optimization Algorithm
Details of problem statement for implementing Steepest Gradient Descent Method is available in the file "Computer Project I description.pdf"


In [4]:
class unconstrained_optimizer:
    def __init__(self):
        return
    
    
    #function is the mathematical function that needs to be optimized
    #gradient is the gadient of the mathematical function.
    #line_search is method of finding alpha
    def Steepest_Gradient_Descent_Method(self,function,gradient,line_search,
                                         linear_first_derivative,condition_check,
                                         epsilon,Iter_max,seed_x,linear_second_derivative=None,condition_param=0.5,seed_alpha=0.5,in_param=None,gamma_param=None):
        #List to store values to be printed at each iteration
        grad_value=[]
        x_value=[]
        alpha_value=[]
        next_x=np.array(seed_x)
        current_alpha_value=seed_alpha
        for i in range(Iter_max):
            #print("Iteration:")
            #print(i)
            current_x=next_x
            current_grad_value=gradient(current_x,in_param,gamma_param)
            x_value.append(current_x)
            grad_value.append(current_grad_value)
            f_of_x=function(current_x,in_param,gamma_param)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.absolute(f_of_x))
            
            if criteria <= epsilon:
                print("Criteria of epsilon met at i:")
                print(i)
                return[x_value,grad_value,alpha_value,criteria]
            if i==25 and condition_param<1e-3:
                condition_param=0.8
                alpha=0.9
            elif i==50 and condition_param<1e-3:
                condition_param=0.8
                alpha=0.9
            elif i==100 and condition_param<1e-2:
                condition_param=0.8
                alpha=0.9
            elif i==200 and condition_param<1e-1:
                condition_param=0.8
                alpha=0.9
            elif i==300:
                alpha=1.5
            elif i==500 :
                condition_param=0.8
                alpha=1.5
            elif i==700:
                alpha=1.5
            
            p=-current_grad_value
            current_alpha_value=line_search(current_x,p,current_alpha_value,function,current_grad_value, 
                                            linear_first_derivative,linear_second_derivative,condition_check,
                                            (condition_param),in_param,gamma_param)
            alpha_value.append(current_alpha_value)
            next_x=current_x+(current_alpha_value*p)
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x=next_x
        current_grad_value=gradient(current_x,in_param,gamma_param)
        x_value.append(current_x)
        grad_value.append(current_grad_value)
        criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))        
        return[x_value,grad_value,alpha_value,criteria]
    
    
    def Newtons_Method(self,function,gradient_function,hessian_function,epsilon,Iter_max,seed_x,in_param=None,gamma_param=None):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.array(seed_x)
        for i in range(Iter_max):
            #print("Iteration :")
            #print(i)
            current_x_value=next_x
            current_grad_value=gradient_function(current_x_value,in_param,gamma_param)
            f_of_x=function(current_x_value,in_param,gamma_param)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.absolute(f_of_x))
            x_value.append(current_x_value)
            grad_value.append(current_grad_value)
            if criteria <= epsilon:
                print("Criteria of epsilon met at i:")
                print(i)
                return[x_value,grad_value,direction_value,criteria]
            current_hessian_value=hessian_function(current_x_value,in_param,gamma_param)
            if np.linalg.det(current_hessian_value)<1e-5:
                print("Warning: Hessian Matrix is Singluar for the current estimate of X!!!!!")
            
            current_direction_value=np.linalg.lstsq(current_hessian_value,current_grad_value)[0]
            direction_value.append(current_direction_value)
            current_hessian_inv=np.linalg.pinv(current_hessian_value)
            next_x=current_x_value-current_hessian_inv.dot(current_direction_value)
            
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x_value=next_x
        current_grad_value=gradient_function(current_x_value,in_param,gamma_param)
        criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))
        x_value.append(current_x_value)
        grad_value.append(current_grad_value)
        return[x_value,grad_value,direction_value,criteria]
    
    def BFGS_Quasi_Newton_Method(self,function,gradient_function,line_search,
                                 linear_first_derivative,condition_check,
                                 epsilon,Iter_max,seed_Hessian,seed_x,
                                 linear_second_derivative=None,condition_param=0.5,seed_alpha=0.5,in_param=None,gamma_param=None):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.matrix(seed_x).T
        next_B=np.matrix(seed_Hessian)
        next_gradient=np.matrix(gradient_function(next_x,in_param,gamma_param)).T
        alpha=seed_alpha
        for i in range(Iter_max):
            current_x_value=next_x
            current_B=next_B
            current_grad_value=next_gradient
            x_value.append(current_x_value)
            grad_value.append(current_grad_value)
            f_of_x=function(current_x_value,in_param,gamma_param)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.absolute(f_of_x))
            if criteria <= epsilon:
                print("Criteria of epsilon met at i:")
                print(i)
                return[x_value,grad_value,direction_value,criteria]
            if np.linalg.det(current_B)<1e-5:
                print("Warning:B matrix is Singluar for the current estimate of X!!!!!")
            
            current_direction_value=np.linalg.lstsq(np.squeeze(np.asarray(current_B)),-1*np.squeeze(np.asarray(current_grad_value)))[0]
            current_direction_value=np.matrix(current_direction_value).T
            direction_value.append(current_direction_value)
            alpha=line_search(current_x_value,current_direction_value,alpha,function,current_grad_value, 
                                            linear_first_derivative,linear_second_derivative,condition_check,
                                            condition_param,in_param,gamma_param)
            next_x=current_x_value+(alpha*current_direction_value)
            next_gradient=np.matrix(gradient_function(next_x,in_param,gamma_param)).T
            s_k=next_x-current_x_value
            y_k=next_gradient-current_grad_value
            next_B=current_B-((((current_B*s_k)*(current_B*s_k).T)/(s_k.T*current_B*s_k)))+((y_k*y_k.T)/(y_k.T*s_k))
            
            
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x_value=next_x
        current_grad_value=gradient_function(current_x_value,in_param,gamma_param)
        criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))
        x_value.append(current_x_value)
        grad_value.append(current_grad_value)
        return[x_value,grad_value,direction_value,criteria]
     

    def Conjugate_Gradient_Descent_Method(self,function,gradient_function,hessian_function,
                                          epsilon,Iter_max,seed_x,in_param=None,gamma_param=None):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.matrix(seed_x).T
        next_gradient=np.matrix(gradient_function(next_x,in_param,gamma_param)).T
        next_direction=-1*next_gradient
        for i in range(Iter_max):
            current_x=next_x
            current_gradient=next_gradient
            current_direction=next_direction
            x_value.append(current_x)
            grad_value.append(current_gradient)
            direction_value.append(current_direction)
            f_of_x=function(current_x,in_param,gamma_param)
            criteria=np.linalg.norm(current_gradient,2)/(1+np.absolute(f_of_x))
            
            if criteria <= epsilon:
                print("Criteria of epsilon met at i:")
                print(i)
                return[x_value,grad_value,direction_value,criteria]
           
            alpha=-1*np.asscalar((current_direction.T*current_gradient)/(current_direction.T*np.matrix(hessian_function(current_x,in_param,gamma_param))*current_direction))
            next_x=current_x+alpha*current_direction
            next_gradient=np.matrix(gradient_function(next_x,in_param,gamma_param)).T
            beta=np.asscalar((current_direction.T*next_gradient)/(current_direction.T*np.matrix(hessian_function(next_x,in_param,gamma_param))*current_direction))
            next_direction=-1*next_gradient+(beta*current_direction)
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x=next_x
        current_gradient=next_gradient
        current_direction=next_direction
        criteria=np.linalg.norm(current_gradient,2)/(1+np.linalg.norm(current_gradient,1))
        x_value.append(current_x)
        grad_value.append(current_gradient)
        direction_value.append(current_direction)
        return[x_value,grad_value,direction_value,criteria]
    

def Newton_line_search(x,p,alpha,function,grad_val,linear_first_derivative,linear_second_derivative,condition_check,condition_param,in_param,gamma_param):
    next_alpha=alpha
    condition=condition_check(condition_param,function,grad_val,x,p,next_alpha,in_param,gamma_param)
    while(condition==False):
        current_alpha=next_alpha
        first_deri=linear_first_derivative(current_alpha,x,p,in_param,gamma_param)
        second_deri=linear_second_derivative(current_alpha,x,p,in_param,gamma_param)
        next_alpha=current_alpha-(first_deri/second_deri)
        condition=condition_check(condition_param,function,grad_val,x,p,next_alpha,in_param,gamma_param)
        
    return next_alpha

#def secant_line_search(x,p,alpha,function,linear_first_derivative,linear_second_derivative,condition_check,condition_param):
#    condition=False
#    next_alpha=alpha
#    while(condition==False):
#        current_alpha=next_alpha
#        first_deri=linear_first_derivative(current_alpha,x,p)
#        second_deri=linear_second_derivative(current_alpha,x,p)
#        next_alpha=current_alpha-(first_deri/second_deri)
#        condition=condition_check(condition_param,function,x,p,next_alpha)
#        
#    return next_alpha   

#def Wolfe_Condition(condition_param,function,x,p,next_alpha):
#    pass

def Armijo_Condition(condition_param,function,grad_val,x,p,alpha,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    grad_val=np.squeeze(np.asarray(grad_val))
    LHS=function(x+alpha*p,in_param,gamma_param)
    RHS=function(x,in_param,gamma_param)+(condition_param*alpha*np.dot(p,grad_val))
    if LHS<=RHS:
        return True
    else:
        return False
    


#### 2.Function 1

In [5]:
def function_1_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=3:
        sys.exit("fuction1 can handle only vector of size 3")
    else:
        return x[0]**2+x[1]**2+x[2]**2

def grad_function_1_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=3:
        print(len(x))
        sys.exit("fuction1 can handle only vector of size 3")
    else:
        return np.array([2*x[0],2*x[1],2*x[2]])
    
def hessian_function_1_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=3:
        sys.exit("fuction1 can handle only vector of size 3")
    else:
        return np.array([[2,0,0],[0,2,0],[0,0,2]])

def linear_first_derivative_function_1_alpha(alpha,x,p,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    return 2*np.dot(x+alpha*p,p) 

def linear_second_derivative_function_1_alpha(alpha,x,p,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    return 2*np.dot(p,p)

optimize=unconstrained_optimizer()
print("SGD Method:")
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_1_x,grad_function_1_x,
                                                                          Newton_line_search,linear_first_derivative_function_1_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([1,1,1]),linear_second_derivative_function_1_alpha,
                                                                          0.5,0.5)
print(xval[0])
print(xval[1])
print()
print("Newton's Method:")
[x_val,grad_val,dir_val,crite]=optimize.Newtons_Method(function_1_x,grad_function_1_x,hessian_function_1_x,1e-5,1000,np.array([1,1,1]))


for x in x_val:
    print(x)
print()
print("BFGS Method:")
[x_val,grad_val,dir_val,crite]=optimize.BFGS_Quasi_Newton_Method(function_1_x,grad_function_1_x,
                                                                          Newton_line_search,linear_first_derivative_function_1_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([[1,0,0],[0,1,0],[0,0,1]]),np.array([1,1,1]),linear_second_derivative_function_1_alpha,
                                                                          0.5,0.5)

print(np.squeeze(np.asarray(x_val[0])))
print(np.squeeze(np.asarray(x_val[1])))
print()
print("CGD Method:")
[x_val,grad_val,dir_val,crite]=optimize.Conjugate_Gradient_Descent_Method(function_1_x,grad_function_1_x,hessian_function_1_x,
                                                                          1e-5,1000,np.array([1,1,1]))
print(np.squeeze(np.asarray(x_val[0])))
print(np.squeeze(np.asarray(x_val[1])))
print()

SGD Method:
Criteria of epsilon met at i:
1
[1 1 1]
[0. 0. 0.]

Newton's Method:
Criteria of epsilon met at i:
19
[1 1 1]
[0.5 0.5 0.5]
[0.25 0.25 0.25]
[0.125 0.125 0.125]
[0.0625 0.0625 0.0625]
[0.03125 0.03125 0.03125]
[0.015625 0.015625 0.015625]
[0.0078125 0.0078125 0.0078125]
[0.00390625 0.00390625 0.00390625]
[0.00195312 0.00195312 0.00195312]
[0.00097656 0.00097656 0.00097656]
[0.00048828 0.00048828 0.00048828]
[0.00024414 0.00024414 0.00024414]
[0.00012207 0.00012207 0.00012207]
[6.10351562e-05 6.10351562e-05 6.10351562e-05]
[3.05175781e-05 3.05175781e-05 3.05175781e-05]
[1.52587891e-05 1.52587891e-05 1.52587891e-05]
[7.62939453e-06 7.62939453e-06 7.62939453e-06]
[3.81469727e-06 3.81469727e-06 3.81469727e-06]
[1.90734863e-06 1.90734863e-06 1.90734863e-06]

BFGS Method:
Criteria of epsilon met at i:
1
[1 1 1]
[0. 0. 0.]

CGD Method:
Criteria of epsilon met at i:
1
[1 1 1]
[0. 0. 0.]



/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:129: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#### 3.Function 2

In [6]:
def function_2_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return x[0]**2+2*x[1]**2-2*x[0]*x[1]-2*x[1]

def grad_function_2_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([2*x[0]-2*x[1],4*x[1]-2*x[0]-2])

def hessian_function_2_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    return np.array([[2,-2],[-2,4]])

def linear_first_derivative_function_2_alpha(alpha,x,p,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    nx=x+alpha*p
    return 2*nx[0]*p[0]+4*nx[1]*p[1]-2*p[0]*nx[1]-2*p[1]*nx[0]-2*p[1]

def linear_second_derivative_function_2_alpha(alpha,x,p,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    return 2*p[0]**2+4*p[1]**2-4*p[1]*p[0]

optimize=unconstrained_optimizer()
print("SGD Method:")
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_2_x,grad_function_2_x,
                                                                          Newton_line_search,linear_first_derivative_function_2_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([0,0]),linear_second_derivative_function_2_alpha,
                                                                          0.01,1)
print("First 10 output:")
for i in range(10):
    print(xval[i])
print()
print("Last 5 output:")
for i in range(48,53):
    print(xval[i])
print()
print("Newton's Method:")
[x_val,grad_val,dir_val,crite]=optimize.Newtons_Method(function_2_x,grad_function_2_x,hessian_function_2_x,1e-5,1000,np.array([0,0]))

print("First 10 output:")
for i in range(10):
    print(x_val[i])
print()
print("Last 5 output:")
for i in range(48,53):
    print(x_val[i])
print()
print("BFGS Method:")
[x_val,grad_val,dir_val,crite]=optimize.BFGS_Quasi_Newton_Method(function_2_x,grad_function_2_x,
                                                                          Newton_line_search,linear_first_derivative_function_2_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([[1,0],[0,1]]),np.array([0,0]),linear_second_derivative_function_2_alpha,
                                                                          0.5,0.5)
print("First 10 output:")
for i in range(10):
    print(np.squeeze(np.asarray(x_val[i])))
print()
print("Last 5 output:")
for i in range(35,40):
    print(np.squeeze(np.asarray(x_val[i])))
print()
print("CGD Method:")
[x_val,grad_val,dir_val,crite]=optimize.Conjugate_Gradient_Descent_Method(function_2_x,grad_function_2_x,hessian_function_2_x,
                                                                          1e-5,1000,np.array([0,0]))
print("First 10 output:")
for i in range(10):
    print(np.squeeze(np.asarray(x_val[i])))
print()
print("Last 5 output:")
for i in range(29,34):
    print(np.squeeze(np.asarray(x_val[i])))
print()

SGD Method:
Criteria of epsilon met at i:
52
First 10 output:
[0 0]
[0.  0.5]
[0.25 0.5 ]
[0.375 0.625]
[0.5    0.6875]
[0.59375 0.75   ]
[0.671875 0.796875]
[0.734375  0.8359375]
[0.78515625 0.8671875 ]
[0.82617188 0.89257812]

Last 5 output:
[0.99995528 0.99997236]
[0.99996382 0.99997764]
[0.99997073 0.99998191]
[0.99997632 0.99998537]
[0.99998084 0.99998816]

Newton's Method:
Criteria of epsilon met at i:
54
First 10 output:
[0 0]
[1.5 1. ]
[1.   0.75]
[1.125 0.875]
[1.0625 0.875 ]
[1.0625  0.90625]
[1.046875 0.921875]
[1.0390625 0.9375   ]
[1.03125    0.94921875]
[1.02539062 0.95898437]

Last 5 output:
[1.00000652 0.99998944]
[1.00000528 0.99999146]
[1.00000427 0.99999309]
[1.00000345 0.99999441]
[1.00000279 0.99999548]

BFGS Method:
Criteria of epsilon met at i:
39
First 10 output:
[0 0]
[0.  0.5]
[0.25  0.625]
[0.4375  0.71875]
[0.578125  0.7890625]
[0.68359375 0.84179688]
[0.76269531 0.88134766]
[0.82202148 0.91101074]
[0.86651611 0.93325806]
[0.89988708 0.94994354]

Last 5 outp

/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:129: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#### 4.Function 3

In [7]:
def function_3_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return (100*(x[1]-x[0]**2)**2)+((1-x[0])**2)

def grad_function_3_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([200*(x[1]-x[0]**2)*(-2*x[0])+(-2*(1-x[0])),200*(x[1]-x[0]**2)])

def hessian_function_3_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([[(1200*x[0]**2)-(400*x[1])+2,-400*x[0]],[-400*x[0],200]])

def linear_first_derivative_function_3_alpha(alpha,x,p,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    nx=x+alpha*p
    return 200*(nx[1]-nx[0]**2)*(p[1]-2*nx[0]*p[0])-2*(1-nx[0])*p[0]

def linear_second_derivative_function_3_alpha(alpha,x,p,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    nx=x+alpha*p
    return 200*(p[1]-2*nx[0]*p[0])*(-2*p[0]**2)+2*p[0]**2

optimize=unconstrained_optimizer()
print("SGD Method:")
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_3_x,grad_function_3_x,
                                                                          Newton_line_search,linear_first_derivative_function_3_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([-1.2,1]),linear_second_derivative_function_3_alpha,
                                                                          1e-4,1e-3)
print("First 10 output:")
for i in range(10):
    print(xval[i])
print()
print("Last 5 output:")
for i in range(995,1000):
    print(xval[i])
print()
print("Newtons Method:")
[x_val,grad_val,dir_val,crite]=optimize.Newtons_Method(function_3_x,grad_function_3_x,hessian_function_3_x,1e-5,1000,np.array([-1.2,1]))

print("First 10 output:")
for i in range(10):
    print(x_val[i])
print()
print("Last 5 output:")
for i in range(995,1000):
    print(x_val[i])
print()
print("BFGS Method")
[x_val,grad_val,dir_val,crite]=optimize.BFGS_Quasi_Newton_Method(function_3_x,grad_function_3_x,
                                                                          Newton_line_search,linear_first_derivative_function_3_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([[1,0],[0,1]]),np.array([-1.2,1]),linear_second_derivative_function_3_alpha,
                                                                          1e-2,1e-2)
print("First 10 output:")
for i in range(10):
    print(np.squeeze(np.asarray(x_val[i]))) 
print()
print("Last 5 output:")
for i in range(995,1000):
    print(np.squeeze(np.asarray(x_val[i]))) 

print()
print("CGD Method")
[x_val,grad_val,dir_val,crite]=optimize.Conjugate_Gradient_Descent_Method(function_3_x,grad_function_3_x,hessian_function_3_x,
                                                                          1e-5,1000,np.array([-1.2,1]))
print("First 10 output:")
for i in range(10):
    print(np.squeeze(np.asarray(x_val[i]))) 
print()
print("Last 5 output:")
for i in range(425,430):
    print(np.squeeze(np.asarray(x_val[i]))) 

print()

SGD Method:
Maximum iteration reached.Criteria of epsilon not met
First 10 output:
[-1.2  1. ]
[-0.9844  1.088 ]
[-1.02727157  1.06420867]
[-1.02688307  1.06242431]
[-1.02608882  1.06083722]
[-1.02531149  1.05924143]
[-1.02453275  1.05764587]
[-1.02375338  1.05605017]
[-1.02297336  1.05445433]
[-1.02219268  1.05285837]

Last 5 output:
[0.3225118  0.10092289]
[0.32346803 0.10154108]
[0.32442122 0.10215918]
[0.3253714  0.10277717]
[0.32631858 0.10339505]

Newtons Method:


/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Maximum iteration reached.Criteria of epsilon not met
First 10 output:
[-1.2  1. ]
[-1.20499381  1.01388852]
[-1.20999516  1.02783251]
[-1.2150038   1.04183165]
[-1.2200195   1.05588561]
[-1.22504202  1.06999405]
[-1.23007109  1.08415659]
[-1.23510647  1.09837285]
[-1.2401479   1.11264241]
[-1.24519513  1.12696485]

Last 5 output:
[-1.75778101  2.86823682]
[-1.75740386  2.86694888]
[-1.75702663  2.86566094]
[-1.75664931  2.86437301]
[-1.75627191  2.86308508]

BFGS Method


/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:129: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Maximum iteration reached.Criteria of epsilon not met
First 10 output:
[-1.2  1. ]
[1.25713082 2.00291054]
[1.55212035 1.99723008]
[1.5520811  1.99721896]
[1.55205484 1.99721947]
[1.55203072 1.9972279 ]
[1.5520086  1.99724359]
[1.55198834 1.99726592]
[1.5519698 1.9972943]
[1.55195285 1.99732815]

Last 5 output:
[1.55023605 2.07123219]
[1.55023428 2.07129879]
[1.55023252 2.07136538]
[1.55023076 2.07143194]
[1.55022899 2.07149849]

CGD Method
Criteria of epsilon met at i:
430
First 10 output:
[-1.2  1. ]
[-1.05669744  1.05849084]
[-1.03089399  1.06894906]
[-1.02258946  1.06208219]
[-1.02571487  1.05823057]
[-1.01765258  1.05177035]
[-1.02069853  1.04789679]
[-1.01284874  1.04179685]
[-1.015822    1.03789777]
[-1.00816112  1.03212084]

Last 5 output:
[0.99840926 0.99642664]
[0.99825528 0.99650659]
[0.99987998 0.9997977 ]
[0.99989527 0.99979012]
[0.99999004 0.99998294]



#### 5.Function 4

In [8]:
def function_4_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return (x[0]+x[1])**4 + x[1]**2

def grad_function_4_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([4*(x[0]+x[1])**3,4*(x[0]+x[1])**3+ 2*x[1]])
    
def hessian_function_4_x(x,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([[12*(x[0]+x[1])**2,12*(x[0]+x[1])**2],[12*(x[0]+x[1])**2,12*(x[0]+x[1])**2+2]])

def linear_first_derivative_function_4_alpha(alpha,x,p,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    nx=x+alpha*p
    return (4*((nx[0]+nx[1])**3)*(p[0]+p[1]))+2*nx[1]*p[1]

def linear_second_derivative_function_4_alpha(alpha,x,p,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    nx=x+alpha*p
    return (12*((nx[0]+nx[1])**2)*(p[0]+p[1])**2)+2*p[1]**2

optimize=unconstrained_optimizer()
print("SGD Method:")
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_4_x,grad_function_4_x,
                                                                          Newton_line_search,linear_first_derivative_function_4_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([2,-2]),linear_second_derivative_function_4_alpha,
                                                                          0.5,10)
print("First 10 output:")
for i in range(10):
    print(xval[i])
print()
print("Last 5 output:")
for i in range(995,1000):
    print(xval[i])
print()
print("Newtons Method:")
[x_val,grad_val,dir_val,crite]=optimize.Newtons_Method(function_4_x,grad_function_4_x,hessian_function_4_x,1e-5,1000,np.array([2,-2]))

print("First 10 output:")
for i in range(10):
    print(x_val[i])
print()
print("Last 5 output:")
for i in range(995,1000):
    print(x_val[i])
print()
print("BFGS Method:")
[x_val,grad_val,dir_val,crite]=optimize.BFGS_Quasi_Newton_Method(function_4_x,grad_function_4_x,
                                                                          Newton_line_search,linear_first_derivative_function_4_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([[1,0],[0,1]]),np.array([2,-2]),linear_second_derivative_function_4_alpha,
                                                                          1e-5,0.5)
print("First 10 output:")
for i in range(10):
    print(np.squeeze(np.asarray(x_val[i])))
print()
print("Last 5 output:")
for i in range(37,42):
    print(np.squeeze(np.asarray(x_val[i])))

print()

print("CGD Method:")
[x_val,grad_val,dir_val,crite]=optimize.Conjugate_Gradient_Descent_Method(function_4_x,grad_function_4_x,hessian_function_4_x,
                                                                          1e-5,1000,np.array([2,-2]))

print("First 10 output:")
for i in range(10):
    print(np.squeeze(np.asarray(x_val[i])))
print()
print("Last 5 output:")
for i in range(374,379):
    print(np.squeeze(np.asarray(x_val[i])))

print()


SGD Method:
Maximum iteration reached.Criteria of epsilon not met
First 10 output:
[ 2 -2]
[ 2.         -1.11238142]
[ 1.86426414 -1.14016317]
[ 1.79057341 -1.10320363]
[ 1.72753768 -1.05917593]
[ 1.66958811 -1.01433486]
[ 1.61498178 -0.97050228]
[ 1.56302493 -0.92827407]
[ 1.51338567 -0.88782643]
[ 1.46587175 -0.84917878]

Last 5 output:
[ 0.05411242 -0.00031683]
[ 0.0540822 -0.0003163]
[ 0.05405203 -0.00031577]
[ 0.05402192 -0.00031524]
[ 0.05399185 -0.00031472]

Newtons Method:


/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Maximum iteration reached.Criteria of epsilon not met
First 10 output:
[ 2 -2]
[ 2. -1.]
[0.72222222 0.16666667]
[0.72706887 0.14814815]
[0.71372632 0.1458695 ]
[0.70046602 0.14326597]
[0.68695824 0.140622  ]
[0.67319521 0.13793004]
[0.65916209 0.13518754]
[0.64484267 0.1323916 ]

Last 5 output:
[7.21869364 1.44405927]
[7.21735776 1.44379215]
[7.21602164 1.44352499]
[7.21468527 1.44325777]
[7.21334865 1.44299051]

BFGS Method:
Criteria of epsilon met at i:
42
First 10 output:
[ 2 -2]
[ 2.   -0.64]
[ 0.92941096 -0.11492916]
[ 0.84205814 -0.07869163]
[ 0.73979205 -0.05867656]
[ 0.65057501 -0.0379798 ]
[ 0.57657208 -0.0263396 ]
[ 0.51210554 -0.0176988 ]
[ 0.456268   -0.01205257]
[ 0.40729671 -0.00815999]

Last 5 output:
[ 1.99657607e-02 -1.53681716e-07]
[ 1.79402301e-02 -1.04196351e-07]
[ 1.61201974e-02 -7.06452261e-08]
[ 1.44848118e-02 -4.78975310e-08]
[ 1.30153386e-02 -3.24745718e-08]

CGD Method:
Criteria of epsilon met at i:
379
First 10 output:
[ 2 -2]
[2. 0.]
[ 1.66670174 -0.3266323

/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:129: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#### 6.Function 5

In [10]:
nc=100
def function_5_x(x,in_param,gamma_param,c=nc):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return (x[0]-1)**2+(x[1]-1)**2+c*(x[0]**2+x[1]**2-0.25)**2

def grad_function_5_x(x,in_param,gamma_param,c=nc):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([(2*(x[0]-1))+(2*c*(x[0]**2+x[1]**2-0.25)*2*x[0]),(2*(x[1]-1))+(2*c*(x[0]**2+x[1]**2-0.25)*2*x[1])])

def hessian_function_5_x(x,in_param,gamma_param,c=nc):
    x=np.squeeze(np.asarray(x))
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([[2+2*c*(x[0]**2+x[1]**2-0.25)*2+8*c*x[0]**2,8*c*x[0]*x[1]],[8*c*x[0]*x[1],2+2*c*(x[0]**2+x[1]**2-0.25)*2+8*c*x[1]**2]])


def linear_first_derivative_function_5_alpha(alpha,x,p,in_param,gamma_param,c=nc):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    nx=x+alpha*p
    return 2*nx[0]*p[0]+2*nx[1]*p[1]+(2*c*(nx[0]**2+nx[1]**2-0.25)*(2*nx[0]*p[0]+2*nx[1]*p[1]))

def linear_second_derivative_function_5_alpha(alpha,x,p,in_param,gamma_param,c=nc):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    nx=x+alpha*p
    return 2*p[0]**2+2*p[1]**2+(2*c*(2*nx[0]*p[0]+2*nx[1]*p[1])**2)+(2*c*(nx[0]**2+nx[1]**2-0.25)*2*p[0]**2+2*p[1]**2)

optimize=unconstrained_optimizer()
print("SGD Method:")
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_5_x,grad_function_5_x,
                                                                          Newton_line_search,linear_first_derivative_function_5_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([1,-1]),linear_second_derivative_function_5_alpha,
                                                                          1e-4,0.5)

print("First 10 output:")
for i in range(10):
    print(xval[i])
print()
print("Last 5 output:")
for i in range(858,863):
    print(xval[i])

print()
print("Newtons Method:")
[x_val,grad_val,dir_val,crite]=optimize.Newtons_Method(function_5_x,grad_function_5_x,hessian_function_5_x,1e-5,1000,np.array([1,-1]))

print("First 10 output:")
for i in range(10):
    print(x_val[i])
print()
print("Last 5 output:")
for i in range(995,1000):
    print(x_val[i])

print()
print("BFGS Method:")
[x_val,grad_val,dir_val,crite]=optimize.BFGS_Quasi_Newton_Method(function_5_x,grad_function_5_x,
                                                                          Newton_line_search,linear_first_derivative_function_5_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([[1,0],[0,1]]),np.array([1,-1]),linear_second_derivative_function_5_alpha,
                                                                          1e-4,0.5)
print("First 10 output:")
for i in range(10):
    print(np.squeeze(np.asarray(x_val[i])))
print()
print("Last 5 output:")
for i in range(995,1000):
    print(np.squeeze(np.asarray(x_val[i])))
print()
print("CGD Method:")
[x_val,grad_val,dir_val,crite]=optimize.Conjugate_Gradient_Descent_Method(function_5_x,grad_function_5_x,hessian_function_5_x,
                                                                          1e-5,1000,np.array([1,-1]))
print("First 10 output:")
for i in range(10):
    print(np.squeeze(np.asarray(x_val[i])))
print()
print("Last 5 output:")
for i in range(218,223):
    print(np.squeeze(np.asarray(x_val[i])))

SGD Method:
Criteria of epsilon met at i:
863
First 10 output:
[ 1 -1]
[-0.79868803  0.80896625]
[ 0.06621275 -0.05672049]
[ 0.08750801 -0.06542138]
[ 0.11360936 -0.07595373]
[ 0.14517635 -0.08848303]
[ 0.18256011 -0.1029966 ]
[ 0.22542662 -0.11914251]
[ 0.27226863 -0.13604422]
[ 0.32004521 -0.15220974]

Last 5 output:
[0.35979201 0.35978705]
[0.35979197 0.35978709]
[0.35979194 0.35978712]
[0.35979191 0.35978716]
[0.35979187 0.35978719]

Newtons Method:


/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Maximum iteration reached.Criteria of epsilon not met
First 10 output:
[ 1 -1]
[ 0.99987159 -0.99986347]
[ 0.99974316 -0.99972692]
[ 0.99961472 -0.99959035]
[ 0.99948626 -0.99945377]
[ 0.99935779 -0.99931717]
[ 0.99922931 -0.99918055]
[ 0.99910081 -0.99904391]
[ 0.9989723  -0.99890726]
[ 0.99884378 -0.99877059]

Last 5 output:
[ 0.86485971 -0.85412128]
[ 0.86471592 -0.85396292]
[ 0.86457211 -0.85380453]
[ 0.86442829 -0.85364611]
[ 0.86428445 -0.85348767]

BFGS Method:


/home/kushal/anaconda3/envs/ece505p1/lib/python3.7/site-packages/ipykernel_launcher.py:129: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Maximum iteration reached.Criteria of epsilon not met
First 10 output:
[ 1 -1]
[-0.79868803  0.80896625]
[-0.79578635  0.8088569 ]
[-0.90324348  0.62290556]
[-0.90242128  0.62261102]
[-0.90174242  0.62218398]
[-0.90107125  0.62174509]
[-0.90040137  0.62130572]
[-0.89973219  0.62086674]
[-0.89906366  0.62042822]

Last 5 output:
[-0.47693426  0.38712272]
[-0.47666554  0.38707661]
[-0.47639695  0.3870309 ]
[-0.47612847  0.3869856 ]
[-0.47586012  0.3869407 ]

CGD Method:
Criteria of epsilon met at i:
223
First 10 output:
[ 1 -1]
[ 0.69591488 -0.69417725]
[ 0.50865067 -0.50361441]
[ 0.40716523 -0.39541478]
[ 0.36972144 -0.34503546]
[ 0.3752828  -0.29890963]
[ 0.41855431 -0.2967335 ]
[ 0.41042529 -0.26440717]
[ 0.46322189 -0.24232753]
[ 0.44538977 -0.21613861]

Last 5 output:
[0.35979179 0.35978741]
[0.35979158 0.35978739]
[0.35979156 0.35978763]
[0.35979137 0.35978761]
[0.35979135 0.35978783]


## 7.Application (Optional)
Refer to the Report PDF for more details.
### Expectation-Maximization Algorithm 
[EM Algorithm](http://www.rmki.kfki.hu/~banmi/elte/bishop_em.pdf)  (Pages 438 and 439 has been used for reference) is part of MLE used when the marginal probabilietes of data are hard to be optimized

##### Assumption Cancerous and Non Cancerous pixels are independent i.e $\rho$ is zero.


step1:  Initialize parameters with seed value.

step2:  Find Expection $P(Z=z/x;\theta)$.

step3: Maximize Q with respect to parameters or Minimize Q w.r.t parameters

step4: check for convergence of parameters of log likelihood function if yes exit else continue


In [31]:
def gaussian_pdf(inp,mean,variance):
    return (1/(2*np.pi*np.sqrt(variance)))*np.exp(-1*((inp-mean)**2/(2*variance)))

def marginal_log_likelihood_funtion_x(x,theta):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P1=theta[4]
        P2=1-theta[4]
        gauss1=gaussian_pdf(x,mu1,v1)
        gauss2=gaussian_pdf(x,mu2,v2)
        return np.sum(np.log(P1*gauss1+P2*gauss2))

def Expection_function(x,theta):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P=[theta[4],1-theta[4]]
        gauss=[gaussian_pdf(x,mu1,v1),gaussian_pdf(x,mu2,v2)]
        denominator=P[0]*gauss[0]+P[1]*gauss[1]
        return [P[0]*gauss[0]/denominator,P[1]*gauss[1]/denominator]

def Q_function(theta,x,gamma):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P1=theta[4]
        P2=1-theta[4]
        gauss1=gaussian_pdf(x,mu1,v1)
        gauss2=gaussian_pdf(x,mu2,v2)
        Q=np.log(P1*gauss1)*gamma[0]+np.log(P2*gauss2)*gamma[1]
        Q=-1*np.sum(Q)
        return Q
        
def grad_Q_function(theta,x,gamma):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P1=theta[4]
        P2=1-theta[4]
        dq_dmu1=-1*np.sum(((x-mu1)/v1)*gamma[0])
        dq_dv1=-1*np.sum(((-0.5/v1)+(0.5*(x-mu1)**2/v1**2))*gamma[0])
        dq_dmu2=-1*np.sum(((x-mu2)/v2)*gamma[1])
        dq_dv2=-1*np.sum(((-0.5/v2)+(0.5*(x-mu2)**2/v2**2))*gamma[1])
        dq_dp=-1*np.sum((1/P1)*gamma[0]-(1/P2)*gamma[1])
        return np.array([dq_dmu1,dq_dv1,dq_dmu2,dq_dv2,dq_dp])

def hessian_Q_function(theta,x,gamma):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P1=theta[4]
        P2=1-theta[4]
        d2q_dmu1=-1*np.sum(((-1/v1)*gamma[0]))
        d2q_dmu1v1=-1*np.sum((-1*(x-mu1)/v1**2)*gamma[0])
        d2q_dv1=-1*np.sum(((0.5/v1**2)+(-1*(x-mu1)**2/v1**3))*gamma[0])
        d2q_dmu2=-1*np.sum((-1/v2)*gamma[1])
        d2q_dmu2v2=-1*np.sum((-1*(x-mu2)/v2**2)*gamma[1])
        d2q_dv2=-1*np.sum(((0.5/v2**2)+(-1*(x-mu2)**2/v2**3))*gamma[1])
        d2q_dp=-1*np.sum((-1/P1**2)*gamma[0]-(-1/P2**2)*gamma[1])
        return np.array([[d2q_dmu1,d2q_dmu1v1,0,0,0],[d2q_dmu1v1,d2q_dv1,0,0,0],[0,0,d2q_dmu2,d2q_dmu2v2,0],[0,0,d2q_dmu2v2,d2q_dv2,0],[0,0,0,0,d2q_dp]])

def linearfd_Q_func_alpha(alpha,theta,theta_p,x,gamma_param):
    theta=np.squeeze(np.asarray(theta))
    theta_p=np.squeeze(np.asarray(theta_p))
    nt=theta+alpha*theta_p
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=nt[0]
        mu2=nt[2]
        v1=nt[1]
        v2=nt[3]
        P1=nt[4]
        P2=1-P1
        rval=((theta_p[4]/P1)+(0.5*(theta_p[1]/v1))+(0.5*(v1*2*(x-mu1)*-1*theta_p[0]-(x-mu1)**2*theta_p[1])/v1**2))*gamma_param[0]
        rval+=((-1*theta_p[4]/P2)+(0.5*(theta_p[3]/v2))+(0.5*(v2*2*(x-mu2)*-1*theta_p[2]-(x-mu2)**2*theta_p[3])/v2**2))*gamma_param[1]
        rval=-1*np.sum(rval)
        return rval
        
def linearsd_Q_func_alpha(alpha,theta,theta_p,x,gamma_param):
    theta=np.squeeze(np.asarray(theta))
    theta_p=np.squeeze(np.asarray(theta_p))
    nt=theta+alpha*theta_p
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=nt[0]
        mu2=nt[2]
        v1=nt[1]
        v2=nt[3]
        P1=nt[4]
        P2=1-P1
        n1=v1*2*(x-mu1)*-1*theta_p[0]-(x-mu1)**2*theta_p[1]
        n2=v2*2*(x-mu2)*-1*theta_p[2]-(x-mu2)**2*theta_p[3]
        dn1=(v1*2*theta_p[0]**2)
        dn2=(v2*2*theta_p[2]**2)
        rval=((-1*theta_p[4]**2/P1**2)+(0.5*(-1*theta_p[1]**2/v1**2))+(-1*(v1**2*dn1-n1*2*v1*theta_p[1])/v1**3))*gamma_param[0]
        rval+=((theta_p[4]**2/P2**2)+(0.5*(-1*theta_p[3]**2/v2**2))+(-1*(v2**2*dn2-n2*2*v2*theta_p[3])/v2**3))*gamma_param[1]
        rval=-1*np.sum(rval)
        return rval
    

In [32]:
def E_M_Algorithm(file_name,seed_theta):
    x=np.loadtxt(file_name,dtype=int,delimiter='\n')
    optimize=unconstrained_optimizer()
    seed_theta=np.array(seed_theta)
    inital_p_x_given_theta=marginal_log_likelihood_funtion_x(x,seed_theta)
    
    if len(seed_theta)!=5:
        sys.exit("E_M can handle only vector of size 5")
   
    for i in range(100):
        # Expectation step
        gamma=Expection_function(x,seed_theta)
        
        # Maximization Step/ minimization step
            
            
        [new_theta,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(Q_function,grad_Q_function,
                                                                         Newton_line_search,linearfd_Q_func_alpha,
                                                                         Armijo_Condition,
                                                                          1e-5,1000,seed_theta,linearsd_Q_func_alpha,
                                                                          1e-4,1e-3,x,gamma)
        
        
        new_p_x_given_theta=marginal_log_likelihood_funtion_x(x,new_theta[-1])
        seed_theta=new_theta[-1]
        c=np.sqrt((new_p_x_given_theta-inital_p_x_given_theta)**2)
        if(c<=1e-5):
            print("Convergence of Marginal probability has happened")
            return seed_theta
        else:
            inital_p_x_given_theta=new_p_x_given_theta
    
    print("Convergence of Marginal probability has not happened")
    return seed_theta
            
        
        
        
    
theta=E_M_Algorithm("image_samples.txt",[120.3091,185.1229,201.7558,201.440,0.61739])
#theta=E_M_Algorithm("image_samples.txt",[100,50,150,50,0.5])
print("Mean1:")
print(theta[0])
print("Var1:")
print(theta[1])
print("Mean2:")
print(theta[2])
print("Var2:")
print(theta[3])
print("P:")
print(theta[4])

Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Cr